In [1]:
import os
import json
from collections import Counter

import torch
from torch.utils.data import Dataset

from src.config import Config
from src.preprocess.data import Preprocess
from src.preprocess.data import AdamWarmup
from src.preprocess.data import ChatDataset

In [2]:
self = Preprocess()
self.get_chat_data()

Read Conversational Data:
  tokenize all the words...
  stemming the words...
  create train & test data...
  export the data into c:\Users\kewjs\Documents\02-Self_Learning\01-Data_Science\07-Chatbot\data\preprocess...
  done creating X and Y data for conversational data using time: 0.03s


In [3]:
self

Size of X-train data: (48, 86), Size of y-train data: (48,)

In [4]:
len(self.all_words)

86

In [14]:
dataset = ChatDataset(x_train=self.X_train, y_train=self.y_train)

In [16]:
train_loader = torch.utils.data.DataLoader(Dataset(),
                                           batch_size=100, 
                                           pin_memory=True)



## Cornel Movies

In [23]:
corpus_movie_conv = os.path.join(self.FILES["CORNELL_MOVIE_DIR"], "{}.txt".format(self.FILES["RAW_CORNELL_CONV"]))
corpus_movie_lines = os.path.join(self.FILES["CORNELL_MOVIE_DIR"], "{}.txt".format(self.FILES["RAW_CORNELL_LINES"]))
max_len = 25

In [24]:
with open(corpus_movie_conv, 'r') as c:
    conv = c.readlines()

with open(corpus_movie_lines, 'r') as l:
    lines = l.readlines()

In [25]:
lines_dic = {}
for line in lines:
    objects = line.split(" +++$+++ ")
    lines_dic[objects[0]] = objects[-1]

In [26]:
def remove_punc(string):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in string:
        if char not in punctuations:
            no_punct = no_punct + char  # space is also a character
    return no_punct.lower()

In [27]:
pairs = []
for con in conv:
    ids = eval(con.split(" +++$+++ ")[-1])
    for i in range(len(ids)):
        qa_pairs = []
        
        if i==len(ids)-1:
            break
        
        first = remove_punc(lines_dic[ids[i]].strip())
        second = remove_punc(lines_dic[ids[i+1]].strip())
        qa_pairs.append(first.split()[:max_len])
        qa_pairs.append(second.split()[:max_len])
        pairs.append(qa_pairs)

In [34]:
pairsdd

[[['can',
   'we',
   'make',
   'this',
   'quick',
   'roxanne',
   'korrine',
   'and',
   'andrew',
   'barrett',
   'are',
   'having',
   'an',
   'incredibly',
   'horrendous',
   'public',
   'break',
   'up',
   'on',
   'the',
   'quad',
   'again'],
  ['well',
   'i',
   'thought',
   'wed',
   'start',
   'with',
   'pronunciation',
   'if',
   'thats',
   'okay',
   'with',
   'you']],
 [['well',
   'i',
   'thought',
   'wed',
   'start',
   'with',
   'pronunciation',
   'if',
   'thats',
   'okay',
   'with',
   'you'],
  ['not',
   'the',
   'hacking',
   'and',
   'gagging',
   'and',
   'spitting',
   'part',
   'please']],
 [['not',
   'the',
   'hacking',
   'and',
   'gagging',
   'and',
   'spitting',
   'part',
   'please'],
  ['okay',
   'then',
   'how',
   'bout',
   'we',
   'try',
   'out',
   'some',
   'french',
   'cuisine',
   'saturday',
   'night']],
 [['youre',
   'asking',
   'me',
   'out',
   'thats',
   'so',
   'cute',
   'whats',
   'your',
   

In [28]:
word_freq = Counter()
for pair in pairs:
    word_freq.update(pair[0])
    word_freq.update(pair[1])

In [29]:
min_word_freq = 5
words = [w for w in word_freq.keys() if word_freq[w] > min_word_freq]
word_map = {k: v + 1 for v, k in enumerate(words)}
word_map['<unk>'] = len(word_map) + 1
word_map['<start>'] = len(word_map) + 1
word_map['<end>'] = len(word_map) + 1
word_map['<pad>'] = 0


In [68]:
print("Total words are: {}".format(len(word_map)))

Total words are: 18243


In [69]:
fname = os.path.join(Config.FILES["PREPROCESS_DIR"], "WORDMAP_corpus.json")
with open(fname, "w") as j:
    json.dump(word_map, j)

In [70]:
def encode_question(words, word_map):
    enc_c = [word_map.get(word, word_map['<unk>']) for word in words] + [word_map['<pad>']] * (max_len - len(words))
    return enc_c


def encode_reply(words, word_map):
    enc_c = [word_map['<start>']] + [word_map.get(word, word_map['<unk>']) for word in words] +     [word_map['<end>']] + [word_map['<pad>']] * (max_len - len(words))
    return enc_c

In [71]:
pairs_encoded = []
for pair in pairs:
    qus = encode_question(pair[0], word_map)
    ans = encode_reply(pair[1], word_map)
    pairs_encoded.append([qus, ans])

In [73]:
fname = os.path.join(Config.FILES["PREPROCESS_DIR"], "pairs_encoded.json")
with open(fname, "w") as p:
    json.dump(pairs_encoded, p)